In [84]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

import sys
sys.path.append('c:/Users/tomas/Documents/Programación/Github/Patricionog/Modulio/Readmes')
import Appio as ap
import Archivio as ac
import Databasio as bd
import Datetimio as dt
import Dictio as dc
import Excelio as ex
import Footio as ft
import Foragio as fg
import Framio as fr
import Graphio as gr
import Highlightio as hg
import Inputio as ip
import Listio as ls
import Mathio as mt
import Notio as nt
import Numbio as nb
import Numpio as nm
import Stringio as st
import Timio as tm
import Tkintio as tn
import Toolio as to

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy import stats

Los métodos robustos disminuyen el grado en que los resultados pueden verse influenciados por violaciones de los supuestos estadísticos clásicos.

In [85]:
# Cargar df.
df = pd.read_excel('J:/My Drive/Experimento/Plantillas finales/1. Generales - Original.xlsx')  

In [86]:
# Definir variables.
Variables_Independientes = ['genero',
            'edad',
            #'nacionalidad',
            #'provincia',
            #'e_social',
            #'niv_educativo',
            'voto_2019',
            #'voto_PASO_2023',
            #'candidato_PASO_2023',
            'autopercep_izqder',
            'autopercep_conpro',
            'autopercep_perantiper',
            'indice_positividad',
            #'massa_ip_izqder',
            #'massa_ip_conpro',
            #'bullrich_ip_izqder',
            #'bullrich_ip_conpro',
            #'schiaretti_ip_izqder',
            #'schiaretti_ip_conpro',
            #'milei_ip_izqder',
            #'milei_ip_conpro',
            #'bregman_ip_izqder',
            #'bregman_ip_conpro',
            'indice_progresismo',
            'indice_conservadurismo'
            ]
Variables_Dependientes = ['cambio_op_con', 'cambio_op_pro', 
                          'cambio_op_con_izq', 'cambio_op_con_der', 'cambio_op_pro_izq', 'cambio_op_pro_der',]

In [87]:
# Rellenar valores NaN con 0 para todo el DataFrame.
df.fillna(0, inplace=True)

In [88]:
# Cortar df.
df = df[Variables_Dependientes + Variables_Independientes]

In [89]:
for Column in Variables_Independientes:
    if df[Column].dtype == 'object':
        df = fr.Create_Dummy_Variables(df, Column, Drop_First = False, Group_Others = True, Remove_Others = False, 
                                    Threshold = 0.05, Name_Other_Column = f"{Column}_Others", Name_Columns_Style = None, Separator=None)

In [90]:
# Object to numeric df.
def Convert_Object_Columns_In_Numerics(df):
    # Convertir columnas booleanas a numéricas (0 y 1).
    for Column in df.columns:
        if df[Column].dtype == 'bool':
            df[Column] = df[Column].astype(int)
    
    return df

df = Convert_Object_Columns_In_Numerics(df)

In [91]:
# Redefinir las variables independientes.
Variables_Independientes = [Column for Column in df.columns if Column not in Variables_Dependientes]
Variables_Independientes.remove('genero_Others')
Variables_Independientes.remove('genero_masculino')
Variables_Independientes.remove('voto_2019_Others')
Variables_Independientes.remove('voto_2019_no vote')
Variables_Independientes.remove('voto_2019_Nicolás Del Caño')

In [92]:
# Generate a list of variables with p-value less than 0.05.
def Generalized_Regression(df, V_Ind, V_Dep, Summary = False):

    # Definir la variable dependiente y las independientes.
    Y = df[V_Dep]
    X = df[V_Ind]  

    # Añadir constante para el término independiente.
    X = sm.add_constant(X)

    # Ajustar el modelo de regresión generalizada con una familia Gaussiana.
    Model = sm.GLM(Y, X, family=sm.families.Gaussian())
    Results = Model.fit()

    # Imprimir resultados.
    if Summary:
        print(Results.summary())

    # Generate a list of variables with p-value less than 0.05.
    Significant_Variables = Results.pvalues[Results.pvalues < 0.05].index.tolist()

    return Significant_Variables

In [93]:
def Check_GLM_Assumptions(DataFrame: pd.DataFrame, V_Ind: list, V_Dep: list):

    """Check the assumptions of Generalized Linear Model (GLM) and print test results."""
    
    Y = DataFrame[V_Dep]
    X = DataFrame[V_Ind]
    
    X = sm.add_constant(X)
    Model = sm.GLM(Y, X, family=sm.families.Binomial()).fit()
    Residuals = Model.resid_deviance
    
    # 1. Linearity of logit
    plt.figure(figsize=(12, 6))

    # Check linearity with respect to continuous predictors
    for Column in X.columns[1:]:  # Skip constant
        sns.regplot(x=X[Column], y=Residuals, lowess=True, line_kws={'color': 'red'})
        plt.title(f'Linearity Check: {Column} vs Residuals')
        plt.xlabel(Column)
        plt.ylabel('Deviance Residuals')
        plt.show()

    # 2. Multicollinearity
    VIF_Data = pd.DataFrame()
    VIF_Data["Feature"] = X.columns
    VIF_Data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    print("\nVariance Inflation Factor (VIF) for Multicollinearity:")
    print(VIF_Data.to_string(index=False))

    # 3. Goodness of Fit - Deviance Test
    print("\nModel Summary:")
    print(Model.summary())

    # 4. Residuals distribution
    plt.figure(figsize=(12, 6))
    
    # Deviance Residuals distribution
    sns.histplot(Residuals, kde=True)
    plt.title('Deviance Residuals Distribution')
    plt.xlabel('Deviance Residuals')
    plt.show()

    # 5. Hosmer-Lemeshow Goodness of Fit Test (for binary GLM)
    # Crear un decil para comparar el modelo predicho con la variable dependiente.
    Groups = pd.qcut(Model.predict(), 10, duplicates='drop')
    Contingency_Table = pd.crosstab(Groups, Y.squeeze())

    # Realizar la prueba de bondad de ajuste Hosmer-Lemeshow.
    H1_Test = sm.stats.diagnostic.linear_harvey_collier(Model, skip=5)

    print("\nHosmer-Lemeshow Test for Goodness of Fit:")
    print(f"Test Statistic: {H1_Test.statistic}, p-value: {H1_Test.pvalue}")

In [94]:
for Variable_Dependiente in Variables_Dependientes:

    V_Dependientes_Para_Comparar = Variables_Independientes

    print('----------------------------------------------------------------')
    print(f'Variable dependiente: {Variable_Dependiente}.')
    print('----------------------------------------------------------------')

    Significativas = Generalized_Regression(df, Variables_Independientes, Variable_Dependiente)

    if 'const' in Significativas:
        Significativas.remove('const')

    #Check_GLM_Assumptions(df, V_Dependientes_Para_Comparar, Variable_Dependiente)

    while sorted(Significativas) != sorted(V_Dependientes_Para_Comparar):
        V_Dependientes_Para_Comparar = Significativas
        Significativas = Generalized_Regression(df, V_Dependientes_Para_Comparar, Variable_Dependiente)

        if 'const' in Significativas:
            Significativas.remove('const')

    print(f'Variables significativas:')
    for Variable in Significativas:
        print(f'Variable: {Variable}')
    #     if sorted(list(df[Variable].unique())) != [0, 1]:
    #         gr.Create_Scatter_Plot(X = df[Variable_Dependiente], Y = df[Variable], X_Label = Variable_Dependiente, Y_Label = Variable)

----------------------------------------------------------------
Variable dependiente: cambio_op_con.
----------------------------------------------------------------
Variables significativas:
Variable: indice_conservadurismo
Variable: voto_2019_Alberto Fernandez
----------------------------------------------------------------
Variable dependiente: cambio_op_pro.
----------------------------------------------------------------
Variables significativas:
Variable: autopercep_izqder
Variable: indice_progresismo
Variable: voto_2019_Alberto Fernandez
----------------------------------------------------------------
Variable dependiente: cambio_op_con_izq.
----------------------------------------------------------------
Variables significativas:
Variable: indice_conservadurismo
----------------------------------------------------------------
Variable dependiente: cambio_op_con_der.
----------------------------------------------------------------
Variables significativas:
Variable: indice_cons

In [95]:
# Suponiendo que df es tu DataFrame y V_Independientes son tus variables independientes.
correlation_matrix = df[Variables_Independientes].corr()
correlation_matrix

,edad,autopercep_izqder,autopercep_conpro,autopercep_perantiper,indice_positividad,indice_progresismo,indice_conservadurismo,genero_femenino,voto_2019_Alberto Fernandez,voto_2019_Mauricio Macri
edad,1.000000,0.021639,-0.091390,-0.005494,0.004671,-0.026496,-0.047310,0.066496,0.098104,0.065136
autopercep_izqder,0.021639,1.000000,-0.419842,0.501464,-0.176342,0.635244,-0.562601,-0.051247,-0.350013,0.482906
autopercep_conpro,-0.091390,-0.419842,1.000000,-0.259513,0.130342,-0.358253,0.405172,0.009479,0.207980,-0.207834
autopercep_perantiper,-0.005494,0.501464,-0.259513,1.000000,-0.322079,0.527083,-0.515143,-0.048240,-0.604151,0.502413
indice_positividad,0.004671,-0.176342,0.130342,-0.322079,1.000000,-0.222230,0.247746,-0.048841,0.280920,-0.198893
indice_progresismo,-0.026496,0.635244,-0.358253,0.527083,-0.222230,1.000000,-0.618557,-0.088793,-0.457871,0.503018
indice_conservadurismo,-0.047310,-0.562601,0.405172,-0.515143,0.247746,-0.618557,1.000000,0.134454,0.450238,-0.442406
genero_femenino,0.066496,-0.051247,0.009479,-0.048240,-0.048841,-0.088793,0.134454,1.000000,0.064872,0.005790
voto_2019_Alberto Fernandez,0.098104,-0.350013,0.207980,-0.604151,0.280920,-0.457871,0.450238,0.064872,1.000000,-0.485236
voto_2019_Mauricio Macri,0.065136,0.482906,-0.207834,0.502413,-0.198893,0.503018,-0.442406,0.005790,-0.485236,1.000000


In [96]:
# Crear un DataFrame con las variables independientes
X = df[Variables_Independientes]
X = sm.add_constant(X)  # Agregar una constante para el modelo

# Calcular el VIF para cada variable
vif = pd.DataFrame()
vif["Variable"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif)

                       Variable         VIF
0                         const  221.666576
1                          edad    1.050294
2             autopercep_izqder    2.057854
3             autopercep_conpro    1.300680
4         autopercep_perantiper    2.044818
5            indice_positividad    1.147753
6            indice_progresismo    2.213360
7        indice_conservadurismo    2.014225
8               genero_femenino    1.040094
9   voto_2019_Alberto Fernandez    1.816391
10     voto_2019_Mauricio Macri    1.665113
